# 東京都立大学のeラーニングシステムのQ&Aデータ¶

## 概要

**データセットを以下の2種類のデータとして再構築する**  

`df_qa`: Queryとその回答に必要な参考情報のID
- Query: 質問
- ID: 回答に必要な参考情報のID

`df_ref`:
- ID: 回答に必要な参考情報のID
Reference: 参考情報

## imports

In [1]:
import os
import copy
import pandas as pd

import warnings
warnings.simplefilter('ignore')

## 1. [Japanese FAQ dataset for e-learning system](https://zenodo.org/record/2783642) からデータをダウンロード

**ダウンロードするデータセット**
- Questions.csv
- Answers.csv 

`参考`

[RAG質問応答システムに使うRetrieverの精度比較](https://www.jiang.jp/posts/20230611_retreiver/) に従って
```python
q_df = pd.read_csv("https://zenodo.org/record/2783642/files/Questions.csv")
a_df = pd.read_csv("https://zenodo.org/record/2783642/files/Answers.csv")
```
で直接データを読み込むことも可能。

## 2. データの読み込み

In [1]:
import pandas as pd
# https://zenodo.org/record/2783642
q_df = pd.read_csv("/Users/macbookpro/Desktop/vector/data/TokyoFAQ/Questions.csv")
a_df = pd.read_csv("/Users/macbookpro/Desktop/vector/data/TokyoFAQ/Answers.csv")
print("q_df.shape:", q_df.shape)
print("a_df.shape:", a_df.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/macbookpro/Desktop/vector/data/TokyoFAQ/Questions.csv'

## 3. 再構築

In [ ]:
q_df.columns = [c.strip() for c in q_df.columns]
a_df.columns = [c.strip() for c in a_df.columns]
df = q_df.merge(a_df, on="AID")
df.columns = ["query","AID","document"]

metadata = a_df[["AID"]].to_dict(orient="records")
documents = a_df["Text"].tolist()
query_list = list(zip(q_df["Text"], q_df["AID"]))

In [92]:
df_qa  = q_df.rename(columns={"Text":"Query","AID":"ID"})
df_qa["Answer"] = pd.NA
df_ref = a_df.rename(columns={"Text":"Reference","AID":"ID"})

In [93]:
df_qa.head(1)

,Query,ID,Answer
0,履修している授業で先生が資料をアップロードしているはずだが、コース上に資料が見当たらない。,A001,<NA>


In [94]:
df_ref.head(1)

,ID,Reference
0,A001,資料が見つからない場合は、以下の点を確認してください。<br><br><br>【受講生編】<...


In [95]:
df_ref.shape,df_ref["ID"].nunique()

((79, 2), 79)

## 4. 保存

In [32]:
dataset_dir = "./dataset/Tokyo"
os.makedirs(dataset_dir,exist_ok=True)
df_ref.to_csv(dataset_dir+"/Reference.csv")
df_qa.to_csv(dataset_dir+"/QA.csv")